In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf

tf.__version__

'2.9.3'

In [4]:
dataset_url = "http://storage.googleapis.com/download.tensorflow.org/data/petfinder-mini.zip"
csv_file = "datasets/petfinder-mini/petfinder-mini.csv"

tf.keras.utils.get_file('petfinder-mini.zip',dataset_url,extract=True,cache_dir='.')
dataframe = pd.read_csv(csv_file)

In [5]:
dataframe.head()

,Type,Age,Breed1,Gender,Color1,Color2,MaturitySize,FurLength,Vaccinated,Sterilized,Health,Fee,Description,PhotoAmt,AdoptionSpeed
0,Cat,3,Tabby,Male,Black,White,Small,Short,No,No,Healthy,100,Nibble is a 3+ month old ball of cuteness. He ...,1,2
1,Cat,1,Domestic Medium Hair,Male,Black,Brown,Medium,Medium,Not Sure,Not Sure,Healthy,0,I just found it alone yesterday near my apartm...,2,0
2,Dog,1,Mixed Breed,Male,Brown,White,Medium,Medium,Yes,No,Healthy,0,Their pregnant mother was dumped by her irresp...,7,3
3,Dog,4,Mixed Breed,Female,Black,Brown,Medium,Short,Yes,No,Healthy,150,"Good guard dog, very alert, active, obedience ...",8,2
4,Dog,1,Mixed Breed,Male,Black,No Color,Medium,Short,No,No,Healthy,0,This handsome yet cute boy is up for adoption....,3,2


In [6]:
dataframe['target'] = np.where(dataframe['AdoptionSpeed']==4,0,1)
dataframe = dataframe.drop(columns=['AdoptionSpeed','Description'])

In [7]:
#podział DataFrame na trzy podstawowe strukturty: zbiór treningowy, walidacyjny i testowy
train,val,test = np.split(dataframe.sample(frac=1),[int(0.8*len(dataframe)),int(0.9*len(dataframe))])

In [8]:
print(f'zbiór treningowy: {len(train)}')
print(f'zbiór walidacyjny: {len(val)}')
print(f'zbiór testowy: {len(test)}')

zbiór treningowy: 9229
zbiór walidacyjny: 1154
zbiór testowy: 1154


In [13]:
#utworzenie potoku wejściowego
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels = df.pop('target')
    df = {key:value[:,tf.newaxis] for key,value in dataframe.items()}
    ds = tf.data.Dataset.from_tensor_slices((dict(df),labels))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(dataframe))
    ds = ds.batch(batch_size)
    ds = ds.prefetch(batch_size)
    return ds

In [14]:
batch_size = 5
train_ds = df_to_dataset(train,batch_size = batch_size)

<ipython-input-13-a989639dc007>:5: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key:value[:,tf.newaxis] for key,value in dataframe.items()}


In [15]:
[(train_features, label_batch)] = train_ds.take(1)
print(f'każdy przypadek: {list(train_features.keys())}')
print(f'Wartość -> Wiek: {train_features["Age"]}')
print(f'Batch: {label_batch}')

każdy przypadek: ['Type', 'Age', 'Breed1', 'Gender', 'Color1', 'Color2', 'MaturitySize', 'FurLength', 'Vaccinated', 'Sterilized', 'Health', 'Fee', 'PhotoAmt', 'target']
Wartość -> Wiek: [[ 6]
 [60]
 [ 5]
 [ 3]
 [10]]
Batch: [1 1 0 1 0]
